In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [5]:
data_dir = Path.cwd().parent / "data" / "merged"
datos = "donantes_final-merged.csv"
data = data_dir / datos
np.random.seed(42)
df = pd.read_csv(data)
df.sample(5)

,Numero,Nombre,Tipo,Contacto,Correo Electronico,Telefono,Observaciones,Razon Social,Tipo de Contribuyente,CUIT,Alta,Baja,Activo,Frecuencia,Importe,Nro de Cuenta,Pais,Mes
109,D00122,El Parque SRL,Empresa,Contacto ASD,parque@mail.com,(+)591 5550-9224,Observación,SRL,Responsable Inscripto,24-78912356-7,2018-01-01,NaN,SI,Semestral,66227.0,403103,Bolivia,Enero
10,D00011,Zenith Digital,Empresa,Contacto K,Zenith Digital@example.com,(011) 5000-0011,Observación 11,S.R.L,Responsable Inscripto,20-12345679-0,2024-03-16,2023-06-14,NO,Bimestral,152834.0,403101,Argentina,Marzo
184,D00111,Futuro Corporativo,Empresa,Contacto E,futurocorporativo@mail.com,(+)595 5380-3426,Observación D,SAS,Monotributista,27-23783266-6,2019-02-12,NaN,SI,Mensual,333984.0,403103,Paraguay,Febrero
77,D00076,Mariana Fernández,Individuo,Contacto XXX,Mariana Fernández@example.com,(011) 5000-0076,Observación 76,SAS,Responsable Inscripto,25-67890130-3,2024-05-18,NaN,SI,Mensual,106340.0,402102,Argentina,Mayo
538,D00127,Horizonte Ecológico,ONG,Contacto AWW,juan.rodriguez@horizonteecologico.com,(+)52 55 1234 5678,Observación DRF,ONG,Monotributista,23-45678901-3,2024-04-01,NaN,SI,Mensual,351798.0,405100,México,Abril


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Numero                 608 non-null    object 
 1   Nombre                 608 non-null    object 
 2   Tipo                   604 non-null    object 
 3   Contacto               608 non-null    object 
 4   Correo Electronico     608 non-null    object 
 5   Telefono               608 non-null    object 
 6   Observaciones          607 non-null    object 
 7   Razon Social           608 non-null    object 
 8   Tipo de Contribuyente  608 non-null    object 
 9   CUIT                   608 non-null    object 
 10  Alta                   608 non-null    object 
 11  Baja                   134 non-null    object 
 12  Activo                 608 non-null    object 
 13  Frecuencia             608 non-null    object 
 14  Importe                608 non-null    float64
 15  Nro de

Vamos a imputar esos valores nulos en vez de eliminarlos

In [7]:
# maybe?
# cond = df["Contacto"] in [" ", "-", ""]
# 
cond = df["Contacto"] == " "
cond2 = df["Contacto"] == "-"
cond3 = df["Contacto"] == ""
df[cond | cond2 | cond3]["Contacto"]

169    -
179    -
185    -
193    -
200    -
210    -
215    -
225    -
232    -
243    -
246    -
254    -
Name: Contacto, dtype: object

In [9]:
df.loc[cond | cond2 | cond3, "Contacto"] = np.nan
nans = df.loc[cond | cond2 | cond3, "Contacto"]

In [10]:
df["Contacto"] = df["Contacto"].ffill()

In [11]:
list(nans.index)

[169, 179, 185, 193, 200, 210, 215, 225, 232, 243, 246, 254]

In [12]:
df.iloc[list(nans.index), 3:5]

,Contacto,Correo Electronico
169,Contacto AK,solidario@mail.com
179,Contacto VDS,solidario@mail.com
185,Contacto E,solidario@mail.com
193,Contacto B,solidario@mail.com
200,Contacto GH,solidario@mail.com
210,Contacto VS,solidario@mail.com
215,Contacto E,solidario@mail.com
225,Contacto A,solidario@mail.com
232,Contacto E,solidario@mail.com
243,Contacto VS,solidario@mail.com


Corregir las razone sociales

In [13]:
print(df["Razon Social"].unique())
print(df.replace({"Razon Social": {"SRL": "S.R.L", "SA": "S.A", "SAS": "S.A.S"}})["Razon Social"].unique())

['S.R.L' 'S.A' 'SAS' 'SA' 'SRL' 'GOB' 'ONG']
['S.R.L' 'S.A' 'S.A.S' 'GOB' 'ONG']


In [14]:
# lo confirmamos
df = df.replace({"Razon Social": {"SRL": "S.R.L", "SA": "S.A", "SAS": "S.A.S"}})
df["Razon Social"].unique()

array(['S.R.L', 'S.A', 'S.A.S', 'GOB', 'ONG'], dtype=object)

In [15]:
mask = df["Tipo"].isna()
nans = df.loc[mask, df.columns[:4]]
nans

,Numero,Nombre,Tipo,Contacto
52,D00052,Mónica León,NaN,Contacto ZZ
69,D00068,Viviana Herrera,NaN,Contacto PPP
80,D00079,Ricardo López,NaN,Contacto AAAA
95,D00094,Rodrigo Medina,NaN,Contacto PPPP


In [16]:
col = "Tipo"
df[col] = df[col].ffill()
print(df[col].isna().sum())

0


In [17]:
# listo
df.iloc[list(nans.index), :4]

,Numero,Nombre,Tipo,Contacto
52,D00052,Mónica León,Individuo,Contacto ZZ
69,D00068,Viviana Herrera,Individuo,Contacto PPP
80,D00079,Ricardo López,Individuo,Contacto AAAA
95,D00094,Rodrigo Medina,Individuo,Contacto PPPP


In [18]:
columns = ['Tipo', 'Razon Social', 'Tipo de Contribuyente', 'Frecuencia', 'Pais']
def unique_col(df, cols):
    cols = {col: df[col].unique() for col in cols}
    return cols
df_uc = unique_col(df, columns)
df_uc

{'Tipo': array(['Empresa', 'Individuo', 'Estado', 'Campaña', 'ONG', 'Tecnología',
        'Agroindustria'], dtype=object),
 'Razon Social': array(['S.R.L', 'S.A', 'S.A.S', 'GOB', 'ONG'], dtype=object),
 'Tipo de Contribuyente': array(['Responsable Inscripto', 'Monotributista', 'Exento',
        'IVA Responsable Inscripto'], dtype=object),
 'Frecuencia': array(['Mensual', 'Bimestral', 'Anual', 'Semestral', 'Trimestral'],
       dtype=object),
 'Pais': array(['Chile', 'Uruguay', 'Bolivia', 'Paraguay', 'Brasil', 'Argentina',
        'Colombia', 'México', 'Perú', 'Venezuela'], dtype=object)}

Me olvide de cambiar `IVA Responsable Inscripto` a solo `Responsable Inscripto`  

In [19]:
tipoc = "Tipo de Contribuyente"
cur = df_uc[tipoc][3]
rep = df_uc[tipoc][0]
df[tipoc] = df[tipoc].str.replace(cur, rep)

In [20]:
df_uc = unique_col(df, columns)
df_uc["Tipo de Contribuyente"]

array(['Responsable Inscripto', 'Monotributista', 'Exento'], dtype=object)

In [21]:
def nunique_col(df):
    cols = [(col, df[col].nunique()) for col in df.columns]
    df_t = pd.DataFrame(cols, columns=["Columna", "Unicos"])
    return df_t
df_nuc = nunique_col(df)
df_nuc

,Columna,Unicos
0,Numero,138
1,Nombre,138
2,Tipo,7
3,Contacto,125
4,Correo Electronico,138
5,Telefono,132
6,Observaciones,130
7,Razon Social,5
8,Tipo de Contribuyente,3
9,CUIT,137


---

Para que no haya `CUIT` duplicados ya que hay uno solo, en vez de borrarlo lo cambio.  
Los demas como `contacto` imagino que se pueden duplicar en la realidad. Los telefonos no deberian, pero ya que no son reales dejamos que se dupliquen

In [22]:
# buscamos
df2 = df.drop_duplicates(subset="Numero")
dup = df2[df2["CUIT"].duplicated()]
dup

,Numero,Nombre,Tipo,Contacto,Correo Electronico,Telefono,Observaciones,Razon Social,Tipo de Contribuyente,CUIT,Alta,Baja,Activo,Frecuencia,Importe,Nro de Cuenta,Pais,Mes
499,D00124,Soluciones VerdeVida,ONG,Contacto DSF,ana.solis@verdevida.com,(+)54 9 11 1234 5678,Observación DE,ONG,Monotributista,20-12345678-9,2024-01-02,NaN,SI,Mensual,221969.0,405100,Argentina,Enero


In [23]:
# vemos quienes lo comparten
cui = dup["CUIT"].iloc[0]
df2[df2["CUIT"] == cui]

,Numero,Nombre,Tipo,Contacto,Correo Electronico,Telefono,Observaciones,Razon Social,Tipo de Contribuyente,CUIT,Alta,Baja,Activo,Frecuencia,Importe,Nro de Cuenta,Pais,Mes
0,D00001,TechNova Solutions,Empresa,Contacto A,TechNova Solutions@example.com,(011) 5000-0001,Observación 1,S.R.L,Responsable Inscripto,20-12345678-9,2024-02-26,2023-05-11,NO,Mensual,292732.0,402101,Chile,Febrero
499,D00124,Soluciones VerdeVida,ONG,Contacto DSF,ana.solis@verdevida.com,(+)54 9 11 1234 5678,Observación DE,ONG,Monotributista,20-12345678-9,2024-01-02,NaN,SI,Mensual,221969.0,405100,Argentina,Enero


In [25]:
# buscamos y cambiamos
replaced_str = df.loc[499, "CUIT"].replace('8', '0')
# nos fijamos que no exista ya
print(f"Cuantos tiene este cuit?: {(df['CUIT'] == replaced_str).sum()}")
# reemplazamos
df.loc[499, "CUIT"] = replaced_str
# deberia dar 138
df["CUIT"].nunique()

Cuantos tiene este cuit?: 0


138

Quedaria solo poner los tipos de datos correcto y elegir las columnas a quedarse  
La columna `Alta` va a ser la fecha en que se hizo la donacion

In [26]:
df.rename(columns={"Alta": "Fecha"}, inplace=True)
df.columns

Index(['Numero', 'Nombre', 'Tipo', 'Contacto', 'Correo Electronico',
       'Telefono', 'Observaciones', 'Razon Social', 'Tipo de Contribuyente',
       'CUIT', 'Fecha', 'Baja', 'Activo', 'Frecuencia', 'Importe',
       'Nro de Cuenta', 'Pais', 'Mes'],
      dtype='object')

In [27]:
print(df["Fecha"].dtype)
df["Fecha"] = pd.to_datetime(df["Fecha"])
print(df["Fecha"].dtype)

object
datetime64[ns]


In [28]:
df["Activo"]

0      NO
1      NO
2      SI
3      NO
4      SI
       ..
603    SI
604    SI
605    SI
606    SI
607    SI
Name: Activo, Length: 608, dtype: object

In [29]:
print(df["Activo"].dtype)
# cambiamos a valores booleanos y despues casteamos el tipo
df["Activo"] = df["Activo"].str.upper().map({"SI": True, "NO": False})
print(df["Activo"].dtype)

object
bool


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Numero                 608 non-null    object        
 1   Nombre                 608 non-null    object        
 2   Tipo                   608 non-null    object        
 3   Contacto               608 non-null    object        
 4   Correo Electronico     608 non-null    object        
 5   Telefono               608 non-null    object        
 6   Observaciones          607 non-null    object        
 7   Razon Social           608 non-null    object        
 8   Tipo de Contribuyente  608 non-null    object        
 9   CUIT                   608 non-null    object        
 10  Fecha                  608 non-null    datetime64[ns]
 11  Baja                   134 non-null    object        
 12  Activo                 608 non-null    bool          
 13  Frecu

In [31]:
df_final = df[["Numero", "Nombre", "Tipo",
               "Contacto", "Correo Electronico",
               "Telefono", "Razon Social",
               "Tipo de Contribuyente",
               "CUIT", "Fecha", "Activo",
               "Frecuencia", "Importe",
               "Nro de Cuenta", "Pais"]]
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Numero                 608 non-null    object        
 1   Nombre                 608 non-null    object        
 2   Tipo                   608 non-null    object        
 3   Contacto               608 non-null    object        
 4   Correo Electronico     608 non-null    object        
 5   Telefono               608 non-null    object        
 6   Razon Social           608 non-null    object        
 7   Tipo de Contribuyente  608 non-null    object        
 8   CUIT                   608 non-null    object        
 9   Fecha                  608 non-null    datetime64[ns]
 10  Activo                 608 non-null    bool          
 11  Frecuencia             608 non-null    object        
 12  Importe                608 non-null    float64       
 13  Nro d

## Last rename

In [32]:
unique_col(df_final, columns)

{'Tipo': array(['Empresa', 'Individuo', 'Estado', 'Campaña', 'ONG', 'Tecnología',
        'Agroindustria'], dtype=object),
 'Razon Social': array(['S.R.L', 'S.A', 'S.A.S', 'GOB', 'ONG'], dtype=object),
 'Tipo de Contribuyente': array(['Responsable Inscripto', 'Monotributista', 'Exento'], dtype=object),
 'Frecuencia': array(['Mensual', 'Bimestral', 'Anual', 'Semestral', 'Trimestral'],
       dtype=object),
 'Pais': array(['Chile', 'Uruguay', 'Bolivia', 'Paraguay', 'Brasil', 'Argentina',
        'Colombia', 'México', 'Perú', 'Venezuela'], dtype=object)}

In [33]:
nunique_col(df_final)

,Columna,Unicos
0,Numero,138
1,Nombre,138
2,Tipo,7
3,Contacto,125
4,Correo Electronico,138
5,Telefono,132
6,Razon Social,5
7,Tipo de Contribuyente,3
8,CUIT,138
9,Fecha,105


In [34]:
data_dir = Path.cwd().parent / "data" / "cleaned"
data_clean = data_dir / "donantes-clean.csv"
df_final.to_csv(data_clean, index=False)
!ls {data_dir}

donantes-clean.csv


In [35]:
asd = pd.read_csv(data_clean)
asd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Numero                 608 non-null    object 
 1   Nombre                 608 non-null    object 
 2   Tipo                   608 non-null    object 
 3   Contacto               608 non-null    object 
 4   Correo Electronico     608 non-null    object 
 5   Telefono               608 non-null    object 
 6   Razon Social           608 non-null    object 
 7   Tipo de Contribuyente  608 non-null    object 
 8   CUIT                   608 non-null    object 
 9   Fecha                  608 non-null    object 
 10  Activo                 608 non-null    bool   
 11  Frecuencia             608 non-null    object 
 12  Importe                608 non-null    float64
 13  Nro de Cuenta          608 non-null    int64  
 14  Pais                   608 non-null    object 
dtypes: boo

In [36]:
nunique_col(asd)

,Columna,Unicos
0,Numero,138
1,Nombre,138
2,Tipo,7
3,Contacto,125
4,Correo Electronico,138
5,Telefono,132
6,Razon Social,5
7,Tipo de Contribuyente,3
8,CUIT,138
9,Fecha,105


In [37]:
unique_col(asd, columns)

{'Tipo': array(['Empresa', 'Individuo', 'Estado', 'Campaña', 'ONG', 'Tecnología',
        'Agroindustria'], dtype=object),
 'Razon Social': array(['S.R.L', 'S.A', 'S.A.S', 'GOB', 'ONG'], dtype=object),
 'Tipo de Contribuyente': array(['Responsable Inscripto', 'Monotributista', 'Exento'], dtype=object),
 'Frecuencia': array(['Mensual', 'Bimestral', 'Anual', 'Semestral', 'Trimestral'],
       dtype=object),
 'Pais': array(['Chile', 'Uruguay', 'Bolivia', 'Paraguay', 'Brasil', 'Argentina',
        'Colombia', 'México', 'Perú', 'Venezuela'], dtype=object)}